# Stock NLP exercise (Using Spark dataframe to read, Pandas dataframe to work):
We will use the news data and try to predict its effect on the Dow Jones Industrial Average (Market Index)

### Getting the data

We will work with a pre-prepared dataset in parquet format.
Place the folder ``DatasetStockNews`` to your data lake storage (associated/linked to Synapse) and configure the path accordingly using ``data_lake_account_name`` and ``file_system_name`` below. 

### Getting ``NLTK`` data
We will be using ```NLTK`` for our feature engineering step. For Enterprise security reasons, Synapse does not allow arbitrary shell scripts to run (which is what nltk data download helpers run in background). 
Download the required ``NLTK`` data from a secure environment and place it in a storage layer accesible from Synapse. 
We have already downloaded the ``NLTK`` data you need for this exercise, place it in following folder: ``abfss://<file_system>@<account_name>.dfs.core.windows.net/synapse/workspaces/<workspace_name>/nltk_data/``

### Configure attributes

Configure the following attributes as per your environment

- ``data_lake_account_name``: Your data lake storage account name
- ``file_system_name``: The blob container name within your storage account, containing the `DatasetStockNews`
- ``workspace_name``: The workspace name for your Synapse environment. This is used to build the ``nltk_data`` folder path (see above section)
- ``subscription_id``: Azure subscription ID
- ``resource_group``: the resource group name
- ``workspace_name``: Workspace name for the associated Azure Machine Learning workspace
- ``workspace_region``: Azure region string for the associated Azure Machine Learning workspace
- ``cog_service_subscription_key``: The API key to call cognitive services. If you do not have one already, for the purpose of this exercise, [please create one](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows)
- ``cog_service_location``: The azure location string for the cognitive service resource (e.g: eastus)

In [3]:
import numpy as np
import pandas as pd
from pyspark.sql.types import *
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pyspark import SparkFiles

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


data_lake_account_name = "hlthmlqhke6c66zwp74"
file_system_name = "raw"
synapse_workspace_name = "synapse-ws-hlthml"

subscription_id = "" 
resource_group = "healthml-rs-rg" 
workspace_name = "ml-hlthml" 
workspace_region = "eastus"

#add stopwords from storage
sc.addFile(f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/synapse/workspaces/{synapse_workspace_name}/nltk_data/",True)
#append path to NLTK
nltk.data.path.append(SparkFiles.getRootDirectory() + '/nltk_data')

StatementMeta(smallspark01, 45, 3, Finished, Available)

# Dataset Description:

There are two channels of data provided in this dataset:

- News data: Crawled historical news headlines from Reddit WorldNews Channel (/r/worldnews). They are ranked by reddit users' votes, and only the top 25 headlines are considered for a single date.
(Range: 2008-06-08 to 2016-07-01)

- Stock data: Dow Jones Industrial Average (DJIA) is used to "prove the concept".
(Range: 2008-08-08 to 2016-07-01)

There are three data files in .csv format:

- RedditNews.csv: two columns
The first column is the "date", and second column is the "news headlines".
All news are ranked from top to bottom based on how hot they are.
Hence, there are 25 lines for each date.

- DJIA_table.csv:
Downloaded directly from Yahoo Finance: check out the web page for more info.

- CombinedNewsDJIA.csv:
Combined dataset with 27 columns.
The first column is "Date", the second is "Label", and the following ones are news headlines ranging from "Top1" to "Top25".

- Combined_news_djia.parquet: combined dataset with 27 columns.
The first column is "Date", the second is "Label", and the following ones are news headlines ranging from "Top1" to "Top25". **We will be using this one for the excercise.**

Dataset citation, Kaggle source: https://www.kaggle.com/aaron7sun/stocknews 

### Read dataset into a dataframe:

In [12]:
import pandas as pd

df_stocknews = pd.read_parquet("abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/DatasetStockNews/Combined_news_djia.parquet".format(file_system_name=file_system_name, data_lake_account_name=data_lake_account_name))
df_stocknews

StatementMeta(smallspark01, 45, 12, Finished, Available)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who k

### Clean up the dataframe with missing/"NA" values and do a quick exploratory check
note how the ``display()`` function can be helpful here, explore visual distribution of ``Label`` column by clicking on ``unique`` value and then selecting chart type

In [13]:
df_stocknews.dropna(inplace= True)
display(df_stocknews, summary=True)

StatementMeta(smallspark01, 45, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, fa68a869-0bcc-4a1c-aa1e-c1e1a604e616)

### Split into training and testing dataset(s)

In [14]:
df_train = df_stocknews[df_stocknews['Date'] < '20150101']
df_test = df_stocknews[df_stocknews['Date'] > '20141231']

StatementMeta(smallspark01, 45, 14, Finished, Available)

### Feature engineering: 
- clean the news headlines, remove special charecters
- combine news headlines into one block
- perform word lemmatization
- vectorize the data

In [15]:
# function for cleaning the data
def clean_data(dataset):
    data = dataset.iloc[:,2:27]
    data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)
    return data

# function for combining the headlines of all the columns into single column
def combine_data(data):
    headlines = []
    for i in range(len(data.index)):
        headlines.append(' '.join(str(x) for x in data.iloc[i, :]))
    return headlines

# function to perform lemmatization of the word
def lemmatize_data(data, lemmatizer):
    cleaned_dataset = []
    #add stopwords from storage
    # sc.addFile(f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/synapse/workspaces/synapse-ws-hlthml/nltk_data/",True)
    #append path to NLTK
    # nltk.data.path.append(SparkFiles.getRootDirectory() + '/nltk_data')
    for i in range(len(data)):
        clean_text = data[i].lower()
        clean_text = clean_text.split()
        clean_text = [lemmatizer.lemmatize(word) for word in clean_text if word not in stopwords.words('english')]
        cleaned_dataset.append(' '.join(clean_text))
    return cleaned_dataset

# function to vectorize the data
def vectorize_data(data, cv):
    vectorized_dataset = cv.fit_transform(data)
    return vectorized_dataset

StatementMeta(smallspark01, 45, 15, Finished, Available)

Apply the feature engineering functions on our train and test dataset(s)

In [16]:
# clean train and test data
clean_train_data = clean_data(df_train)
clean_test_data = clean_data(df_test)

# combine the headlines in single column
comb_train_data = combine_data(clean_train_data)
comb_test_data = combine_data(clean_test_data)

# lemmatize data
lemmatizer = WordNetLemmatizer()
train_data = lemmatize_data(comb_train_data, lemmatizer)
test_data = lemmatize_data(comb_test_data, lemmatizer)

# vectorize data
cv = CountVectorizer(ngram_range=(2,2))
vec_train_data = vectorize_data(train_data, cv)
vec_test_data = cv.transform(test_data)

StatementMeta(smallspark01, 45, 16, Finished, Available)

### Train a random forest classifier

In [17]:
# create classifier
rf_clf = RandomForestClassifier(n_estimators=200, criterion='entropy')
rf_clf.fit(vec_train_data, df_train['Label'])

StatementMeta(smallspark01, 45, 17, Finished, Available)

RandomForestClassifier(criterion='entropy', n_estimators=200)

### Check prediction metrices

In [18]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# run precictions on test data
y_pred = rf_clf.predict(vec_test_data)

print(classification_report(df_test['Label'], y_pred))



StatementMeta(smallspark01, 45, 18, Finished, Available)

              precision    recall  f1-score   support

           0       1.00      0.70      0.82       186
           1       0.77      1.00      0.87       192

    accuracy                           0.85       378
   macro avg       0.89      0.85      0.85       378
weighted avg       0.89      0.85      0.85       378